# Speed of Light from Maxwell's Equations

This notebook contains the programmatic verification for the **Speed of Light from Maxwell's Equations** entry from the THEORIA dataset.

**Entry ID:** speed_of_light  
**Required Library:** sympy 1.12.0

## Description
In vacuum, Maxwell's equations imply that disturbances of the electromagnetic field propagate as waves whose speed is fixed by the electric permittivity and magnetic permeability of free space. Combining the curl equations for E and B leads to a wave equation with propagation speed v = 1/sqrt(epsilon_0*mu_0). Identifying this speed with the experimentally measured speed of light shows that light is an electromagnetic wave and fixes the universal constant c from purely electromagnetic measurements.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# Programmatic verification: speed of light from Maxwell's equations
# We check that a wave equation with coefficient mu_0*epsilon_0
# implies a propagation speed v = 1/sqrt(mu_0*epsilon_0), and that
# this is equivalent to writing 1/c^2 = mu_0*epsilon_0.

# Symbols
mu_0, epsilon_0, c, v = sp.symbols('mu_0 epsilon_0 c v', positive=True, real=True)

# Generic wave equation form: nabla^2 F = (1/v^2) * d^2 F / dt^2
# Maxwell in vacuum gives: nabla^2 F = mu_0*epsilon_0 * d^2 F / dt^2

eq_wave_match = sp.Eq(1/v**2, mu_0*epsilon_0)
v_solution = sp.solve(eq_wave_match, v)[0]
assert sp.simplify(v_solution - 1/sp.sqrt(mu_0*epsilon_0)) == 0

# Define c as the propagation speed and check consistency with 1/c^2 = mu_0*epsilon_0
eq_def_c = sp.Eq(c, 1/sp.sqrt(mu_0*epsilon_0))
eq_inv_c2 = sp.Eq(1/c**2, mu_0*epsilon_0)

# Substitute the definition of c into 1/c^2 and verify we recover mu_0*epsilon_0
lhs_sub = eq_inv_c2.lhs.subs(c, eq_def_c.rhs)
assert sp.simplify(lhs_sub - eq_inv_c2.rhs) == 0

print('Speed of light from Maxwell verification passed \u2714')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=speed_of_light.json](https://theoria-dataset.org/entries.html?entry=speed_of_light.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0